<a href="https://colab.research.google.com/github/Bryan-Az/Mathematics-LLM/blob/model_eval/%5BEvaluation%2C_GGUF%2C_Quantization%5D_Mathematics_LLM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating the Math Finetuned 'Education & Math' Pre-Trained HuggingFaceTB SmolLM2-1.7B-Instruct Model as Compared to the Base Finetuned Model

This notebook is running on a GPU environment in Google Colab. The pre-trained foundation model we are using is being pulled from a cloud repository on HuggingFace. The model backbone was originally hosted by HuggingFaceTB, before being fine-tuned on our own dataset of math problems and subsequently uploaded to our own public model repo. Both the finetuned base/pre-trained LLaMA models have been quantized for use with llama.cpp for less memory-intense inference.

## Imports and Installs

In [1]:
# login with huggginface for using gated LLaMA foundational models
!huggingface-cli login --token $hf_token

usage: huggingface-cli <command> [<args>] login [-h] [--token TOKEN] [--add-to-git-credential]
huggingface-cli <command> [<args>] login: error: argument --token: expected one argument


In [2]:
%%capture
!pip install llama-cpp-python
!pip install datasets

In [4]:
from google.colab import userdata
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from torch.utils.data import Dataset as TorchDataset
from datasets import load_dataset
import pandas as pd
import re
import os
hf_token = userdata.get('HF_TOKEN')

## Loading the Quantized Models

In [7]:
finetuned_pretrained_model_repo = 'Alexis-Az/Math-Problem-LlaMA-3.2-1.7B-GGUF'
finetuned_base_model_repo = 'Alexis-Az/Math-Problem-LlaMA-3.2-1B-GGUF'
filename = 'unsloth.Q4_K_M.gguf'
filename_ftpt = 'unsloth_ftpt.Q4_K_M.gguf'
filename_ftb = 'unsloth_ftb.Q4_K_M.gguf'
max_seq_length = 4096

In [8]:
# Download the file
temp_path = hf_hub_download(repo_id=finetuned_base_model_repo, filename=filename, local_dir='.')

# Rename the file
os.rename(temp_path, filename_ftb)
# Download the file
temp_path = hf_hub_download(repo_id=finetuned_pretrained_model_repo, filename=filename, local_dir = '.')

# Rename the file
os.rename(temp_path, filename_ftpt)

unsloth.Q4_K_M.gguf:   0%|          | 0.00/955M [00:00<?, ?B/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [9]:
%%capture
finetuned_base_model = Llama(model_path=filename_ftb, max_seq_length=max_seq_length, verbose=False)

In [10]:
%%capture
finetuned_pretrained_model= Llama(model_path=filename_ftpt, max_seq_length=max_seq_length, verbose=False)

## Loading the Math Related Evaluation Datasets

In [11]:
integration_dataset="Alexis-Az/math_datasets"

### Addition Data

In [12]:
val_additions = (load_dataset(integration_dataset, 'additions', split='test')).shuffle()

README.md:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

addition_operations_train.csv:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

additions/addition_operations_eval.csv:   0%|          | 0.00/2.86M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/801600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200401 [00:00<?, ? examples/s]

In [13]:
val_additions

Dataset({
    features: ['Operation', 'Result'],
    num_rows: 200401
})

### Roots Data

In [14]:
val_roots = (load_dataset(integration_dataset, 'roots', split="train[-2000:]")).shuffle() # the last 2000 rows from the train set were heldout for eval

roots/Roots.csv:   0%|          | 0.00/5.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [15]:
val_roots

Dataset({
    features: ['Function', 'Roots'],
    num_rows: 2000
})

### Derivatives Data

In [16]:
val_derivs = (load_dataset(integration_dataset, 'derivatives', split="train[-2000:]")).shuffle() # the last 2000 rows from the train set were heldout for eval

derivatives/Derivatives.csv:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [18]:
val_derivs

Dataset({
    features: ['Function', 'Derivative'],
    num_rows: 2000
})

## Evaluating the Models on the Eval Datasets

In [19]:
num_problems = 50

In [20]:
def solve_math_problem(example, llm, task):
    if task == 'Additions':
      response = llm(
          f"Solve this math problem, only print the final answer, do not provide steps: {example['Operation']}",
          max_tokens=50
      )
    if task == 'Roots':
      response = llm(
          f"Solve this math problem, only print the final answer, do not provide steps: {example['Function']}",
          max_tokens=50
      )

    if task == 'Derivatives':
      response = llm(
          f"Solve this math problem, only print the final answer, do not provide steps: {example['Function']}",
          max_tokens=50
      )

    solution = response['choices'][0]['text'].strip()
    return solution

def extract_number(text, is_text=False):
    if is_text:
      return text
    else:
      match = re.search(r'\d+', text)
      if match:
          # Always return as string to avoid overflow
          return match.group()
      else:
          return None

def process_example(example, llm, subset, model_name):
    assert(subset in ['Roots', 'Derivatives', 'Additions'])
    assert(model_name in ['BFT', 'PTFT'])
    example[model_name + '_Raw_Result_' + subset] = solve_math_problem(example, llm, subset)
    extracted_result = None
    # Apply extract_number to get the result
    if subset == 'Additions':
      extracted_result = extract_number(example[model_name + '_Raw_Result_' + subset], is_text=False)
    if subset == 'Roots':
      extracted_result = extract_number(example[model_name + '_Raw_Result_' + subset], is_text=True)
    if subset == 'Derivatives':
      extracted_result = extract_number(example[model_name + '_Raw_Result_' + subset], is_text=True)

    # Store result, handling potential string values
    example[model_name + '_Model_Result_' + subset] = extracted_result

    # Compare with 'Result' column, handling potential string values if integer overflow occurs
    if subset == 'Additions':
      example[model_name + '_Model_Result_' + subset +'_Correct'] = str(example['Result']) == str(extracted_result) if extracted_result else False
    if subset == 'Roots':
      example[model_name + '_Model_Result_' + subset +'_Correct'] = str(example['Roots']) == str(extracted_result) if extracted_result else False
    if subset == 'Derivatives':
      example[model_name + '_Model_Result_' + subset +'_Correct'] = str(example['Derivative']) == str(extracted_result) if extracted_result else False

    return example

In [21]:
# sampling 50 rows from the huggingface datasets
additions_eval_subset = val_additions.shuffle().select(range(num_problems))

In [22]:
roots_eval_subset = val_roots.shuffle().select(range(num_problems))

In [23]:
derivatives_eval_subset = val_derivs.shuffle().select(range(num_problems))

### The Base Fine-tuned 1B Model

In [24]:
# Assuming all datasets are a Hugging Face Dataset object
additions_eval_subset = additions_eval_subset.map(lambda example: process_example(example, finetuned_base_model, 'Additions', 'BFT'))

Parameter 'function'=<function <lambda> at 0x7d2e60682c20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [25]:
roots_eval_subset = roots_eval_subset.map(lambda example: process_example(example, finetuned_base_model, 'Roots', 'BFT'))

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [26]:
derivatives_eval_subset = derivatives_eval_subset.map(lambda example: process_example(example, finetuned_base_model, 'Derivatives', 'BFT'))

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

### The Pre-trained & Fine-tuned 1.7B Model

In [27]:
additions_eval_subset = additions_eval_subset.map(lambda example: process_example(example, finetuned_pretrained_model, 'Additions', 'PTFT'))

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [28]:
roots_eval_subset = roots_eval_subset.map(lambda example: process_example(example, finetuned_pretrained_model, 'Roots', 'PTFT'))

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [29]:
derivatives_eval_subset = derivatives_eval_subset.map(lambda example: process_example(example, finetuned_pretrained_model, 'Derivatives', 'PTFT'))

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## Evaluating Additions using MSE/RMSE

In [32]:
# show a sample of the additions eval subset as a pandas table
additions_eval_subset.to_pandas().head()

,Operation,Result,BFT_Raw_Result_Additions,BFT_Model_Result_Additions,BFT_Model_Result_Additions_Correct,PTFT_Raw_Result_Additions,PTFT_Model_Result_Additions,PTFT_Model_Result_Additions_Correct
0,428 + 982,1410,- 476 - 528 - 2 - 142 - 7 - 4 - 3...,476,False,+ 25881 - 1406 - 922 - 421 - 2850 - 980 - 2198...,25881,False
1,480 + 634,1114,x 30 - 2 x 2 x 3 - 2 x 1 + 2 x 3 + 3 x 2 x 1 -...,30,False,6100000000000000000000000000000000000000000000...,6100000000000000000000000000000000000000000000...,False
2,963 + 536,1499,+ 191 - 32 - 24 - 2 - 5 - 11 - 7 - 6 - ...,191,False,2054462451504243201005048048011694704003024250...,2054462451504243201005048048011694704003024250...,False
3,113 + 130,243,x 6 - 9 4 8 x - 2 3 5 6 6 7 9 2 2 ...,6,False,5730000000000000000000000000000000000000000000...,5730000000000000000000000000000000000000000000...,False
4,838 + 957,1795,+ 732 - 838 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 ...,732,False,60 - 1207800 - 1239000000000000000000000000000...,60,False


In [62]:
# Create a new Dataset with the desired columns
columns_to_keep = ['BFT_Model_Result_Additions', 'PTFT_Model_Result_Additions', 'Operation', 'Result']
rmse_subset = additions_eval_subset.from_dict(
    {col: additions_eval_subset[col] for col in columns_to_keep}
)
rmse_subset = rmse_subset.to_pandas()

In [73]:
mse_bft = ((rmse_subset['BFT_Model_Result_Additions'].astype(float) - rmse_subset['Result'].astype(float)) ** 2).mean()
mse_ptft = ((rmse_subset['PTFT_Model_Result_Additions'].astype(float) - rmse_subset['Result'].astype(float)) ** 2).mean()
rmse_bft = mse_bft ** 0.5
rmse_ptft = mse_ptft ** 0.5
print(f"BFT MSE: {mse_bft}, BFT RMSE: {rmse_bft}")
print(f"PTFT MSE: {mse_ptft}, PTFT RMSE: {rmse_ptft}")

BFT MSE: 1067220.42, BFT RMSE: 1033.0636088837898
PTFT MSE: 1.126745931191077e+99, PTFT RMSE: 3.356703637783766e+49


## Saving the Evaluations to a HuggingFace Dataset

In [74]:
evaluations_repo = 'Alexis-Az/Math-LLM-Evaluations'
additions_eval_subset.push_to_hub(evaluations_repo, 'Additions')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Alexis-Az/Math-LLM-Evaluations/commit/07254be26ce2b3347636ff26be080649867a17cc', commit_message='Upload dataset', commit_description='', oid='07254be26ce2b3347636ff26be080649867a17cc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Alexis-Az/Math-LLM-Evaluations', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Alexis-Az/Math-LLM-Evaluations'), pr_revision=None, pr_num=None)

In [75]:
roots_eval_subset.push_to_hub(evaluations_repo, 'Roots', split='test')
derivatives_eval_subset.push_to_hub(evaluations_repo, 'Derivatives', split = 'test')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Alexis-Az/Math-LLM-Evaluations/commit/df88578fabdbfc4e73523120455339dba300c493', commit_message='Upload dataset', commit_description='', oid='df88578fabdbfc4e73523120455339dba300c493', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Alexis-Az/Math-LLM-Evaluations', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Alexis-Az/Math-LLM-Evaluations'), pr_revision=None, pr_num=None)